In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install jovian

     |████████████████████████████████| 68 kB 3.9 MB/s 
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6501 sha256=80ff65ec273342dd865549e077d5a0cbe705ffc1f0982cc9cf81bdefec22193b
  Stored in directory: /root/.cache/pip/wheels/2a/ea/87/dd57f1ecb4f0752f3e1dbf958ebf8b36d920d190425bcdc24d
Successfully built uuid


In [42]:
#library imports
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
import jovian
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

In [4]:
datapath='/content/drive/MyDrive/Text_class/title_classification_data/'

In [5]:
df1=pd.read_csv(datapath+"papers_100K.csv")
df2=pd.read_csv(datapath+"topics.csv")

In [6]:
df=pd.merge(df1, df2)

In [7]:
df.head()

,userkey,paper_title,publisher,clean_research_topics
0,ZXBZUbMAAAAJ,A pH-activatable and aniline-substituted photo...,"Chemical science 6 (10), 5969-5977","cancer therapy,theranostics"
1,ZXBZUbMAAAAJ,A near-infrared reversible fluorescent probe f...,"Chemical Communications 47 (33), 9468-9470","cancer therapy,theranostics"
2,ZXBZUbMAAAAJ,H<sub>2</sub>O<sub>2</sub>-Activatable and O<s...,Journal of the American Chemical Society 137 (...,"cancer therapy,theranostics"
3,ZXBZUbMAAAAJ,Two design strategies for enhancement of multi...,"Chemical science 3 (8), 2587-2597","cancer therapy,theranostics"
4,ZXBZUbMAAAAJ,A Multifunctional Nanomicelle for Real‐Time Ta...,Angewandte Chemie International Edition 53 (36...,"cancer therapy,theranostics"


In [8]:
df.shape

(99999, 4)

In [9]:
df = df[['paper_title', 'clean_research_topics']]
df.head()

,paper_title,clean_research_topics
0,A pH-activatable and aniline-substituted photo...,"cancer therapy,theranostics"
1,A near-infrared reversible fluorescent probe f...,"cancer therapy,theranostics"
2,H<sub>2</sub>O<sub>2</sub>-Activatable and O<s...,"cancer therapy,theranostics"
3,Two design strategies for enhancement of multi...,"cancer therapy,theranostics"
4,A Multifunctional Nanomicelle for Real‐Time Ta...,"cancer therapy,theranostics"


In [10]:
df.columns=['title','topics']

In [11]:
df.head()

,title,topics
0,A pH-activatable and aniline-substituted photo...,"cancer therapy,theranostics"
1,A near-infrared reversible fluorescent probe f...,"cancer therapy,theranostics"
2,H<sub>2</sub>O<sub>2</sub>-Activatable and O<s...,"cancer therapy,theranostics"
3,Two design strategies for enhancement of multi...,"cancer therapy,theranostics"
4,A Multifunctional Nanomicelle for Real‐Time Ta...,"cancer therapy,theranostics"


In [12]:
df['title_length'] = df['title'].apply(lambda x: len(x.split()))

In [13]:
df.head()

,title,topics,title_length
0,A pH-activatable and aniline-substituted photo...,"cancer therapy,theranostics",9
1,A near-infrared reversible fluorescent probe f...,"cancer therapy,theranostics",15
2,H<sub>2</sub>O<sub>2</sub>-Activatable and O<s...,"cancer therapy,theranostics",15
3,Two design strategies for enhancement of multi...,"cancer therapy,theranostics",16
4,A Multifunctional Nanomicelle for Real‐Time Ta...,"cancer therapy,theranostics",12


In [14]:
np.mean(df['title_length'])

16.149351493514935

In [15]:
#tokenization
from gensim.utils import tokenize
list(tokenize(df.title[1]))

['A',
 'near',
 'infrared',
 'reversible',
 'fluorescent',
 'probe',
 'for',
 'peroxynitrite',
 'and',
 'imaging',
 'of',
 'redox',
 'cycles',
 'in',
 'living',
 'cells']

In [16]:
tok = spacy.load("en_core_web_sm")
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [17]:
#count number of occurences of each word
counts = Counter()
for index, row in df.iterrows():
    counts.update(tokenize(row['df'])

SyntaxError: ignored

In [77]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

NameError: ignored

In [78]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [79]:
df['encoded'] = df['title'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


,title,topics,title_length,topics_list,title_tok,encoded
0,A pH-activatable and aniline-substituted photo...,"cancer therapy,theranostics",9,"[cancer therapy, theranostics]","[a, ph, activatable, and, aniline, substituted...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,..."
1,A near-infrared reversible fluorescent probe f...,"cancer therapy,theranostics",15,"[cancer therapy, theranostics]","[a, near, infrared, reversible, fluorescent, p...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
2,H<sub>2</sub>O<sub>2</sub>-Activatable and O<s...,"cancer therapy,theranostics",15,"[cancer therapy, theranostics]","[h, sub, , sub, o, sub, , sub, , activa...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
3,Two design strategies for enhancement of multi...,"cancer therapy,theranostics",16,"[cancer therapy, theranostics]","[two, design, strategies, for, enhancement, of...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
4,A Multifunctional Nanomicelle for Real‐Time Ta...,"cancer therapy,theranostics",12,"[cancer therapy, theranostics]","[a, multifunctional, nanomicelle, for, real, t...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,..."


In [80]:
X = list(df['encoded'])
y = list(df['topics'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [ ]:
class Title_topics(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [81]:
train_ds = Title_topics(X_train, y_train)
valid_ds = Title_topics(X_valid, y_valid)

NameError: ignored

In [ ]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [ ]:
batch_size = 5000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [ ]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [ ]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)

In [ ]:
train_model(model_fixed, epochs=30, lr=0.01)

AttributeError: 'tuple' object has no attribute 'long'

In [ ]:
###############################

In [18]:
# define topics into the list such as deined here is topics_list
df['topics_list'] = df['topics'].apply(lambda x: str(x).split(','))
df['title_tok']=df['title'].apply(lambda x:tokenize(x))

In [69]:
df.head()
df.title[2]

'H<sub>2</sub>O<sub>2</sub>-Activatable and O<sub>2</sub>-Evolving Nanoparticles for Highly Efficient and Selective Photodynamic Therapy against Hypoxic Tumor Cells'

In [20]:
df.topics.iloc[0]

'cancer therapy,theranostics'

In [21]:
df.topics_list.iloc[0]

['cancer therapy', 'theranostics']

In [22]:
df_req=df[['title_tok','topics_list']]
df_req

,title_tok,topics_list
0,"[a, ph, activatable, and, aniline, substituted...","[cancer therapy, theranostics]"
1,"[a, near, infrared, reversible, fluorescent, p...","[cancer therapy, theranostics]"
2,"[h, sub, , sub, o, sub, , sub, , activa...","[cancer therapy, theranostics]"
3,"[two, design, strategies, for, enhancement, of...","[cancer therapy, theranostics]"
4,"[a, multifunctional, nanomicelle, for, real, t...","[cancer therapy, theranostics]"
...,...,...
99994,"[the, flourishing, scale, , fs, , by, diener...","[primary care, health promotion, health scienc..."
99995,"[parkinson, s, disease, , l, dopa, , and, en...","[primary care, health promotion, health scienc..."
99996,"[die, bedeutung, achtsamkeitsbasierter, interv...","[primary care, health promotion, health scienc..."
99997,"[converging, cellular, processes, for, substan...","[primary care, health promotion, health scienc..."


In [23]:
import ast

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
logreg = LogisticRegression(max_iter=1000)

from sklearn.multiclass import OneVsRestClassifier

In [25]:

df_req.head()

,title_tok,topics_list
0,"[a, ph, activatable, and, aniline, substituted...","[cancer therapy, theranostics]"
1,"[a, near, infrared, reversible, fluorescent, p...","[cancer therapy, theranostics]"
2,"[h, sub, , sub, o, sub, , sub, , activa...","[cancer therapy, theranostics]"
3,"[two, design, strategies, for, enhancement, of...","[cancer therapy, theranostics]"
4,"[a, multifunctional, nanomicelle, for, real, t...","[cancer therapy, theranostics]"


In [26]:
y=df_req['topics_list']

In [27]:
#transform y into one hot vectors
multilabel=MultiLabelBinarizer()

In [28]:
y=multilabel.fit_transform(df_req['topics_list'])

In [29]:
len(y[1])

1592

In [30]:
multilabel.classes_

array(['3d bioprinting', '3d printing', '3d scanning', ...,
       'الادارة الرياضية', 'التدريب الرياضى وعلوم الحركة',
       'العلوم التربوية والنفسية والاجتماعية'], dtype=object)

In [31]:
# transform y into data frame with their classes defined in it
pd.DataFrame(y,columns=multilabel.classes_)

,3d bioprinting,3d printing,3d scanning,aav,accounting,acoustic emission,acquisitions,action,acupuncture,addiction,addiction medicine,additive manufacturing,adhd,adipocyte biology,adipogenesis,adipose biology,administrative law,adolescent development,adolescent health,adolescent medicine,adsorption,adult neurogenesis,aerosol science,aging,agricultural economics,agriculture,agroforestry,agronomy,air pollution,air quality,algebra,algorithms,allergy,alliances,als,altruism,ambulatory care,amd,american political development,american politics,...,viral replication,virology,vision,visual cognition,visual neurophysiology,visual search,visualization,waste management,waste to energy,wastewater treatment,water,water desalination,water pollution,water resources,water resources systems,water treatment,wavelets,wearable sensors,web technologies,weeds,wine,wireless communications,wireless networks,wireless sensor networks,wood composites,wood science,wood technology,work,wound healing,wsn,x-ray astronomy,x-ray crystallography,x-ray ct,x-ray diffraction,young stars,zoology,zoonoses,الادارة الرياضية,التدريب الرياضى وعلوم الحركة,العلوم التربوية والنفسية والاجتماعية
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
df_req['title_tok']

0        [a, ph, activatable, and, aniline, substituted...
1        [a, near, infrared, reversible, fluorescent, p...
2        [h, sub,    , sub, o, sub,    , sub,  , activa...
3        [two, design, strategies, for, enhancement, of...
4        [a, multifunctional, nanomicelle, for, real, t...
                               ...                        
99994    [the, flourishing, scale,  , fs,  , by, diener...
99995    [parkinson, s, disease,  , l, dopa,  , and, en...
99996    [die, bedeutung, achtsamkeitsbasierter, interv...
99997    [converging, cellular, processes, for, substan...
99998    [die, neuronale, basis, von, meditation, und, ...
Name: title_tok, Length: 99999, dtype: object

In [33]:
titles=[' '.join( title) for title in df_req.title_tok]

In [34]:
titles[0]

'a ph activatable and aniline substituted photosensitizer for near infrared cancer theranostics'

In [35]:
tfidf=TfidfVectorizer(analyzer='word',max_features=10000)
X=tfidf.fit_transform(titles)

In [36]:
X

<99999x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 1039128 stored elements in Compressed Sparse Row format>

In [37]:
X.shape, y.shape

((99999, 10000), (99999, 1592))

In [38]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=0)

## Build Model

In [50]:
sgd=SGDClassifier()
logreg=LogisticRegression(solver='lbfgs')
svc=LinearSVC()

In [51]:
def j_score(y_test, y_pred):
  jaccard=np.minimum(y_test, y_pred).sum(axis=1)/np.maximum(y_test, y_pred).sum(axis=1)
  return jaccard.mean()*100

def print_score(y_pred, clf):
  print("Clf: ",clf.__class__.__name__)
  print("Jacard score : {}".format(j_score(y_test, y_pred)))
  print('------')

In [52]:
for classifier in [sgd,lr,svc]:
    clf=OneVsRestClassifier(classifier)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    print_score(y_pred,classifier)

/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 343 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 592 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


Clf:  SGDClassifier
Jacard score : 0.6361666666666665
------


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 343 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 592 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Clf:  LogisticRegression
Jacard score : 5.319130952380952
------


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 343 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 592 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


Clf:  LinearSVC
Jacard score : 23.614039682539683
------


**Model test with real data**

In [70]:
x=['Evolving Nanoparticles for Highly Efficient and Selective Photodynamic Therapy against Hypoxic Tumor Cells']

In [71]:
xt=tfidf.transform(x)

In [72]:
clf.predict(xt)

array([[0, 0, 0, ..., 0, 0, 0]])

In [73]:
multilabel.inverse_transform(clf.predict(xt))

[('cancer therapy', 'drug delivery', 'theranostics')]

In [1]:
x=['Thermodynamic of Halogen bonding in solution']

In [2]:
xt=tfidf.transform(x)

NameError: ignored

In [ ]:
# i didn't eat a big pie